In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to the dataset
train_dir = 'chest_xray_org/train'
val_dir = 'chest_xray_org/val'
test_dir = 'chest_xray_org/test'

# Rescale the images
train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images from the directory
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Load the ResNet50 model with pre-trained weights on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build the model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [7]:
# Train the model
history = model.fit(train_generator, epochs=10, validation_data=val_generator, verbose=1)

C:\Users\jhaka\Desktop\BSBI\Courses\Computer Vision\assignment\tfven\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1474s 9s/step - accuracy: 0.8941 - loss: 0.9780 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1485s 9s/step - accuracy: 0.9137 - loss: 0.2017 - val_accuracy: 0.5000 - val_loss: 0.8278
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1495s 9s/step - accuracy: 0.9529 - loss: 0.1316 - val_accuracy: 0.5000 - val_loss: 0.7351
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1462s 9s/step - accuracy: 0.9508 - loss: 0.1556 - val_accuracy: 0.5000 - val_loss: 0.7157
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1563s 10s/step - accuracy: 0.9651 - loss: 0.1037 - val_accuracy: 0.6875 - val_loss: 0.4843
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1597s 10s/step - accuracy: 0.9658 - loss: 0.0881 - val_accuracy: 0.7500 - val_loss: 0.5721
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1583s 10s/step - accuracy: 0.9818 - loss: 0.0615 - val_accuracy: 0.5625 - val_loss: 1.1179
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1618s 10s/step - accuracy: 0.9842 - loss: 0.044

In [11]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Evaluate model on test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy}")

# Make predictions on test set
Y_pred = model.predict(test_generator)
y_pred = (Y_pred > 0.5).astype(int)

# Generate classification report
print(classification_report(test_generator.classes, y_pred, target_names=['Normal', 'Pneumonia']))


20/20 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.6898 - loss: 2.9020
Test Accuracy: 0.7147436141967773
20/20 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step
              precision    recall  f1-score   support

      Normal       0.44      0.12      0.18       234
   Pneumonia       0.63      0.91      0.75       390

    accuracy                           0.61       624
   macro avg       0.53      0.51      0.46       624
weighted avg       0.56      0.61      0.53       624

